In [0]:
---CAMADA BRONZE

In [0]:
CREATE CATALOG IF NOT EXISTS bronze_cyber_breaches;
USE CATALOG bronze_cyber_breaches;
CREATE SCHEMA IF NOT EXISTS bronze_cyber_breaches;
USE SCHEMA bronze_cyber_breaches;

In [0]:
CREATE CATALOG IF NOT EXISTS main;

CREATE SCHEMA IF NOT EXISTS main.bronze;
CREATE SCHEMA IF NOT EXISTS main.silver;
CREATE SCHEMA IF NOT EXISTS main.gold;

In [0]:
USE CATALOG main;
USE SCHEMA bronze;

In [0]:
CREATE VOLUME IF NOT EXISTS main.bronze.raw_data;

In [0]:
USE CATALOG main;
USE SCHEMA bronze;

CREATE OR REPLACE TABLE bronze_cyber_breaches
AS SELECT * 
FROM csv.`/Volumes/main/bronze/raw_data/bronze_cyber_breaches.csv`;

In [0]:
SELECT * FROM bronze_cyber_breaches LIMIT 20;

In [0]:
---CAMADA SILVER

In [0]:
USE CATALOG main;
USE SCHEMA bronze;

DESCRIBE TABLE bronze_cyber_breaches;

In [0]:
USE CATALOG main;
USE SCHEMA silver;

--- Ajustando os nomes das colunas

DROP TABLE IF EXISTS silver_cyber_breaches;

CREATE TABLE silver_cyber_breaches AS
SELECT
    -- ID
    TRY_CAST(_c0 AS INT) AS breach_id,

    -- Organização
    TRIM(_c1) AS organization,

    -- Ano 
    TRY_CAST(_c2 AS INT) AS year,

    -- Records Exposed 
    TRY_CAST(
        REGEXP_REPLACE(_c3, '[^0-9]', '') 
        AS BIGINT
    ) AS records_exposed,

    -- Tipo de organização (texto)
    INITCAP(TRIM(_c4)) AS organization_type,

    -- Método de violação
    INITCAP(TRIM(_c5)) AS breach_method,

    -- Metadados
    current_timestamp() AS silver_load_timestamp

FROM main.bronze.bronze_cyber_breaches;


In [0]:
SELECT * FROM silver_cyber_breaches LIMIT 20;

In [0]:
delete from silver_cyber_breaches where breach_id IS NULL

In [0]:
USE CATALOG main;
USE SCHEMA silver;

SELECT * FROM silver_cyber_breaches LIMIT 20;

In [0]:
DESCRIBE TABLE main.silver.silver_cyber_breaches;

In [0]:
SELECT MAX(TRY_CAST(records_exposed AS BIGINT)) 
FROM main.silver.silver_cyber_breaches;

In [0]:
---- CAMADA GOLD

In [0]:
USE CATALOG main;
USE SCHEMA gold;

In [0]:
---Criando a tabela Dimensão de Organização

DROP TABLE IF EXISTS dim_organization;

CREATE TABLE dim_organization AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY organization) AS organization_key,
    organization
FROM (
    SELECT DISTINCT organization
    FROM main.silver.silver_cyber_breaches
    WHERE organization IS NOT NULL
) sub;

In [0]:
---Criando a tabela Dimensão de Tipo de Organização

DROP TABLE IF EXISTS dim_organization_type;

CREATE TABLE dim_organization_type AS
SELECT
  ROW_NUMBER() OVER (ORDER BY organization_type) AS organization_type_key,
  organization_type
FROM (
  SELECT DISTINCT organization_type
  FROM main.silver.silver_cyber_breaches
  WHERE organization_type IS NOT NULL
) t;

In [0]:
---Criando a tabela Dimensão de Tipo de Ataque

DROP TABLE IF EXISTS dim_breach_method;

CREATE TABLE dim_breach_method AS
SELECT
  ROW_NUMBER() OVER (ORDER BY breach_method) AS breach_method_key,
  breach_method
FROM (
  SELECT DISTINCT breach_method
  FROM main.silver.silver_cyber_breaches
  WHERE breach_method IS NOT NULL
) t;

In [0]:
---Criando a tabela Dimensão Ano

CREATE OR REPLACE TABLE main.gold.dim_year AS
SELECT
    ROW_NUMBER() OVER (ORDER BY year) AS year_key,
    year
FROM (
    SELECT DISTINCT year
    FROM main.silver.silver_cyber_breaches
    WHERE year IS NOT NULL
);

In [0]:
---Criando a Tabela Fato: Cyber Breaches referenciando todas as dimensões

CREATE OR REPLACE TABLE main.gold.fact_cyber_breaches AS
SELECT
    s.breach_id,
    o.organization_key,
    ot.organization_type_key,
    bm.breach_method_key,
    y.year_key,
    s.records_exposed,
    s.silver_load_timestamp AS fact_load_timestamp
FROM main.silver.silver_cyber_breaches s

LEFT JOIN main.gold.dim_organization o
    ON s.organization = o.organization

LEFT JOIN main.gold.dim_organization_type ot
    ON s.organization_type = ot.organization_type

LEFT JOIN main.gold.dim_breach_method bm
    ON s.breach_method = bm.breach_method

LEFT JOIN main.gold.dim_year y
    ON s.year = y.year;

In [0]:
SHOW TABLES IN main.gold;

In [0]:
----- Consultas nas dimensões

SELECT COUNT(*) AS total_dim_organization
FROM main.gold.dim_organization;

SELECT COUNT(*) AS total_dim_organization_type
FROM main.gold.dim_organization_type;

SELECT COUNT(*) AS total_dim_breach_method
FROM main.gold.dim_breach_method;

SELECT COUNT(*) AS total_dim_year
FROM main.gold.dim_year;



In [0]:
SELECT COUNT(*) AS total_dim_breach_method
FROM main.gold.dim_breach_method;






In [0]:
----- Consulta na tabela fato


SELECT COUNT(*) AS total_fact
FROM main.gold.fact_cyber_breaches;

In [0]:

----- Comparativo entre tabelas (Silver e Gold)

SELECT 
    SUM(records_exposed) AS total_records_exposed
FROM main.gold.fact_cyber_breaches;

SELECT 
    SUM(records_exposed)
FROM main.silver.silver_cyber_breaches;


In [0]:
---Ver uma amostra dos dados

SELECT *
FROM main.gold.fact_cyber_breaches
LIMIT 10;



In [0]:
----Teste de integridade (PK e FK)

SELECT COUNT(*) AS fk_organization_null
FROM main.gold.fact_cyber_breaches
WHERE organization_key IS NULL;

SELECT COUNT(*) AS fk_org_type_null
FROM main.gold.fact_cyber_breaches
WHERE organization_type_key IS NULL;

SELECT COUNT(*) AS fk_method_null
FROM main.gold.fact_cyber_breaches
WHERE breach_method_key IS NULL;

SELECT COUNT(*) AS fk_year_null
FROM main.gold.fact_cyber_breaches
WHERE year_key IS NULL;



In [0]:
SELECT organization_key, COUNT(*)
FROM main.gold.dim_organization
GROUP BY organization_key
HAVING COUNT(*) > 1;


In [0]:

---- Como retornou '3' na dim_year e dim_breach_method foi aplicado a correção 
---- Criar registro “Desconhecido” na dimensão 

---  Inserir linha técnica na dim_year
INSERT INTO main.gold.dim_year (year_key, year)
VALUES (-1, -1);

--- Atualizar a fato para apontar para esse registro
UPDATE main.gold.fact_cyber_breaches
SET year_key = -1
WHERE year_key IS NULL;

--- Aplicar constraint
ALTER TABLE main.gold.fact_cyber_breaches
ADD CONSTRAINT chk_year_key_not_null
CHECK (year_key IS NOT NULL);

In [0]:
---- Como retornou '1' na dim_year e dim_breach_method foi aplicado a correção 
---- Criar registro “Desconhecido” na dimensão 

---  Inserir linha técnica na dim_breach_method
INSERT INTO main.gold.dim_breach_method (breach_method_key, breach_method)
VALUES (-1, 'Unknown');

--- Atualizar a tabela fato
UPDATE main.gold.fact_cyber_breaches
SET breach_method_key = -1
WHERE breach_method_key IS NULL;

--- Aplicar constraint
ALTER TABLE main.gold.fact_cyber_breaches
ADD CONSTRAINT chk_breach_method_key_not_null
CHECK (breach_method_key IS NOT NULL);


In [0]:

--- Consultas ---

---- Total de registros comprometidos por ano
SELECT 
    y.year,
    SUM(f.records_exposed) AS total_records
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_year y
    ON f.year_key = y.year_key
GROUP BY y.year
ORDER BY y.year;

---- Total de registros comprometidos método de ataque
SELECT 
    m.breach_method,
    SUM(f.records_exposed) AS total_records
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_breach_method m
    ON f.breach_method_key = m.breach_method_key
GROUP BY m.breach_method
ORDER BY total_records DESC;



In [0]:
SELECT COUNT(*) AS total_nulls
FROM main.gold.fact_cyber_breaches
WHERE records_exposed IS NULL;


In [0]:
--- Tratativas

ALTER TABLE main.gold.fact_cyber_breaches
ADD CONSTRAINT chk_records_exposed_positive
CHECK (records_exposed >= 0 OR records_exposed IS NULL);

In [0]:
---Garantindo que as abelas não estão vazias

SELECT 'fact' AS tabela, COUNT(*) AS total FROM main.gold.fact_cyber_breaches
UNION ALL
SELECT 'dim_organization', COUNT(*) FROM main.gold.dim_organization
UNION ALL
SELECT 'dim_organization_type', COUNT(*) FROM main.gold.dim_organization_type
UNION ALL
SELECT 'dim_breach_method', COUNT(*) FROM main.gold.dim_breach_method
UNION ALL
SELECT 'dim_year', COUNT(*) FROM main.gold.dim_year;

In [0]:
---Consulta FKs não nulas

SELECT
  SUM(CASE WHEN organization_key IS NULL THEN 1 ELSE 0 END) AS org_null,
  SUM(CASE WHEN organization_type_key IS NULL THEN 1 ELSE 0 END) AS type_null,
  SUM(CASE WHEN breach_method_key IS NULL THEN 1 ELSE 0 END) AS method_null,
  SUM(CASE WHEN year_key IS NULL THEN 1 ELSE 0 END) AS year_null
FROM main.gold.fact_cyber_breaches;

In [0]:
SELECT COUNT(*) AS fk_orfa
FROM main.gold.fact_cyber_breaches f
LEFT JOIN main.gold.dim_year d
  ON f.year_key = d.year_key
WHERE d.year_key IS NULL;

In [0]:
--- Teste de consulta analítica

SELECT
  y.year,
  bm.breach_method,
  SUM(f.records_exposed) AS total_records
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_year y ON f.year_key = y.year_key
JOIN main.gold.dim_breach_method bm ON f.breach_method_key = bm.breach_method_key
GROUP BY y.year, bm.breach_method
ORDER BY y.year;

In [0]:

----- ANÁLISE DE DADOS- RESPONDER AS QUESTÕES: 

--1.	Quais são os tipos de ataques mais comuns?
--2.	Por que os ataques às empresas estão aumentando?
--3.	Quais tipos de empresas são mais visadas de ataques?
--4.	Para cada tipo de ataque, qual é a forma mais eficiente de prevenção?
--5.	As análises permitem prever cenários futuros de segurança cibernética? Quais são as perspectivas?



In [0]:
--1.	Quais são os tipos de ataques mais comuns?

SELECT
  bm.breach_method,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_breach_method bm
  ON f.breach_method_key = bm.breach_method_key
GROUP BY bm.breach_method
ORDER BY total_incidentes DESC;


--Frequência absoluta de cada método de ataque
--Identifica os vetores mais recorrentes
--Base para priorização de controles de segurança

-- Interpretação esperada

--Ataques como Hacking, Misconfiguration, Accidentally Published e Insider tendem a aparecer com maior frequência, indicando falhas técnicas e humanas como principais vetores.

Databricks visualization. Run in Databricks to view.

In [0]:
--2.	Por que os ataques às empresas estão aumentando?

SELECT
  y.year,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_year y
  ON f.year_key = y.year_key
WHERE y.year <> -1
GROUP BY y.year
ORDER BY y.year;

--Esta é uma pergunta interpretativa, não é totalmente estatística.

-- O aumento pode ser associado a:

-- maior volume de dados sensíveis armazenados
-- maior exposição pública
-- maior interesse dos atacantes


Databricks visualization. Run in Databricks to view.

In [0]:
--3.	Quais tipos de empresas são mais visadas de ataques?

SELECT
  ot.organization_type,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_organization_type ot
  ON f.organization_type_key = ot.organization_type_key
GROUP BY ot.organization_type
ORDER BY total_incidentes DESC
LIMIT 10;


--Interpretação

--Setores como:

--Government
--Healthcare
--Financial
--Technology
--tendem a concentrar mais incidentes, refletindo:
--valor dos dados
--criticidade dos serviços
--maior visibilidade pública

Databricks visualization. Run in Databricks to view.

In [0]:
--4.	Para cada tipo de ataque, qual é a forma mais eficiente de prevenção?

SELECT
  bm.breach_method,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_breach_method bm
  ON f.breach_method_key = bm.breach_method_key
GROUP BY bm.breach_method;

--Associação técnica (exemplo)
--Tipo de ataque	Medida de prevenção
--Hacking	Hardening, MFA, IDS/IPS
--Misconfiguration	IaC, revisão de configs
--Accidentally Published	DLP, controle de acesso
--Insider	IAM, segregação de funções
--Lost / Stolen	Criptografia, MDM
-- Observação acadêmica:
--As medidas não vêm do dataset, mas de boas práticas de cibersegurança.

In [0]:
--5.	As análises permitem prever cenários futuros de segurança cibernética? Quais são as perspectivas?


SELECT
  y.year,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_year y
  ON f.year_key = y.year_key
WHERE y.year <> -1
GROUP BY y.year
ORDER BY y.year;

--conclusão técnica
--O dataset permite identificar tendências históricas
--Não permite previsão determinística
--Pode apoiar análises exploratórias e projeções simples
--Modelos preditivos exigiriam:
--dados contínuos
--mais variáveis
--séries temporais completas


Databricks visualization. Run in Databricks to view.

In [0]:
--Complemento da pergunta 5 - "As análises permitem prever cenários futuros de segurança cibernética? Quais são as perspectivas? " 
---Evolução dos métodos de ataque ao longo do tempo

SELECT
  y.year,
  bm.breach_method,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_year y
  ON f.year_key = y.year_key
JOIN main.gold.dim_breach_method bm
  ON f.breach_method_key = bm.breach_method_key
WHERE y.year <> -1
GROUP BY y.year, bm.breach_method
ORDER BY y.year, total_incidentes DESC
LIMIT 20;




Databricks visualization. Run in Databricks to view.

In [0]:
SELECT
  y.year,
  COUNT(*) AS total_incidentes
FROM main.gold.fact_cyber_breaches f
JOIN main.gold.dim_year y
  ON f.year_key = y.year_key
WHERE y.year >= 2018
GROUP BY y.year
ORDER BY y.year;


---teste simples